In [10]:
# 검증 전용 (1단계): 배당 공시 데이터에서 stock_code, 공시일 추출
import pandas as pd

# 1️⃣ 데이터 로드
path = "/Users/gun/Desktop/미래에셋 AI 공모전/data/dividend_ml_ready.csv"
df_div = pd.read_csv(path, dtype={"stock_code": str, "rcept_no": str})
print(f"✅ 데이터 로드 완료: {len(df_div):,} rows")

# 2️⃣ 공시일(rcept_dt) 추출 → rcept_no 앞 8자리
df_div["rcept_dt"] = pd.to_datetime(df_div["rcept_no"].str[:8], format="%Y%m%d", errors="coerce")

# 3️⃣ 필요한 컬럼만 정제
df_base = df_div[["stock_code", "rcept_dt"]].dropna().drop_duplicates().reset_index(drop=True)

# 4️⃣ 확인
print(f"✅ 유효한 stock_code + rcept_dt 조합 수: {len(df_base):,}")
print(df_base.head())

# 5️⃣ 저장 (price_fetcher 이후 사용할 수 있도록)
save_path = "/Users/gun/Desktop/미래에셋 AI 공모전/data/base_for_price_check.csv"
df_base.to_csv(save_path, index=False, encoding="utf-8-sig")
print(f"📁 저장 완료: {save_path}")

✅ 데이터 로드 완료: 15,460 rows
✅ 유효한 stock_code + rcept_dt 조합 수: 15,460
  stock_code   rcept_dt
0     900290 2025-06-20
1     900290 2025-03-19
2     900290 2024-12-10
3     900290 2024-11-20
4     900290 2024-11-06
📁 저장 완료: /Users/gun/Desktop/미래에셋 AI 공모전/data/base_for_price_check.csv


In [14]:
# 전체 검증 파이프라인
import pandas as pd
from tqdm.auto import tqdm
from datetime import timedelta
import FinanceDataReader as fdr

# 1️⃣ 기준 테이블 로드 ─────────────────────────────────────
base_path = "/Users/gun/Desktop/미래에셋 AI 공모전/data/base_for_price_check.csv"
df_base = pd.read_csv(base_path, dtype={"stock_code": str})
df_base["rcept_dt"] = pd.to_datetime(df_base["rcept_dt"])
print(f"✅ 기준 데이터 로드: {len(df_base):,} rows")

# 2️⃣ 주가 수집 함수 ─────────────────────────────────────
def fetch_price_history(code: str, start: str, end: str) -> pd.DataFrame:
    """FDR에서 해당 종목코드의 주가 수집"""
    try:
        df = fdr.DataReader(code, start, end)
        df = df.reset_index()[["Date", "Close", "Volume"]]
        df["stock_code"] = code
        df = df.rename(columns={"Date": "date", "Close": "close", "Volume": "volume"})
        return df
    except:
        return pd.DataFrame()

# 3️⃣ 종목별 주가 수집 ─────────────────────────────────────
codes = df_base["stock_code"].unique()
price_all = []

print("📦 주가 수집 시작...")
for code in tqdm(codes):
    start_date = (df_base[df_base["stock_code"] == code]["rcept_dt"].min() - timedelta(days=30)).strftime("%Y-%m-%d")
    end_date   = (df_base[df_base["stock_code"] == code]["rcept_dt"].max() + timedelta(days=30)).strftime("%Y-%m-%d")
    df_price = fetch_price_history(code, start_date, end_date)
    price_all.append(df_price)

df_price_all = pd.concat(price_all, ignore_index=True)
df_price_all["date"] = pd.to_datetime(df_price_all["date"])
print(f"✅ 주가 수집 완료: {len(df_price_all):,} rows")

# 4️⃣ 병합을 위한 컬럼 정제 ───────────────────────────────
df_merge = df_base.merge(df_price_all, on="stock_code", how="left")
df_merge = df_merge[df_merge["date"].notna()]
print(f"🔗 병합 후 데이터 수: {len(df_merge):,}")

# 5️⃣ 윈도우 추출 함수 ─────────────────────────────────────
def select_trading_window(df_group: pd.DataFrame,
                          dt: pd.Timestamp,
                          window: int = 10) -> pd.DataFrame:
    dfg = df_group.sort_values("date").reset_index(drop=True)
    pos = dfg["date"].searchsorted(dt)
    start = max(pos - window, 0)
    end   = min(pos + window + 1, len(dfg))
    return dfg.iloc[start:end]

# 6️⃣ 종목별 공시일 ±10거래일 윈도우 확보 여부 검증 ─────
records = []
grouped = df_merge.groupby(["stock_code", "rcept_dt"])
print("🔍 그룹 수:", grouped.ngroups)

for (code, dt), grp in tqdm(grouped, total=grouped.ngroups, desc="Validating windows"):
    if grp.empty:
        n = 0
    else:
        win = select_trading_window(grp, dt, window=10)
        n = len(win)
    records.append({
        "stock_code": code,
        "rcept_dt":   dt,
        "n_days":     n
    })

df_check = pd.DataFrame(records)
print("✅ 검증 결과 생성:", df_check.shape)

# 7️⃣ 결과 확인 ───────────────────────────────────────────
dist = df_check["n_days"].value_counts().sort_index()
print("\n±10거래일 윈도우 확보 분포 (n_days):")
print(dist.to_string())

# 8️⃣ 부족 이벤트 출력 ───────────────────────────────────
missing = df_check[df_check["n_days"] < 21]
print(f"\n▶ 윈도우 부족 이벤트 수: {len(missing):,}")
if not missing.empty:
    print("\n-- 부족 예시 (최초 5건) --")
    print(missing.head().to_string(index=False))

# 9️⃣ 선택 저장 (필요 시)
save_path = "/Users/gun/Desktop/미래에셋 AI 공모전/data/window_check_result.csv"
df_check.to_csv(save_path, index=False, encoding="utf-8-sig")
print(f"📁 검증 결과 저장 완료: {save_path}")

✅ 기준 데이터 로드: 15,460 rows
📦 주가 수집 시작...


100%|██████████| 1682/1682 [05:56<00:00,  4.72it/s]


✅ 주가 수집 완료: 752,906 rows
🔗 병합 후 데이터 수: 7,638,780
🔍 그룹 수: 4022


Validating windows: 100%|██████████| 4022/4022 [00:00<00:00, 5901.53it/s]

✅ 검증 결과 생성: (4022, 3)

±10거래일 윈도우 확보 분포 (n_days):
n_days
11      92
12       1
13       1
16       1
19       1
20       4
21    3922

▶ 윈도우 부족 이벤트 수: 100

-- 부족 예시 (최초 5건) --
stock_code   rcept_dt  n_days
    100120 2013-02-08      11
    100130 2013-03-04      11
    100220 2013-02-27      11
    100250 2013-02-07      11
    100660 2013-02-28      11
📁 검증 결과 저장 완료: /Users/gun/Desktop/미래에셋 AI 공모전/data/window_check_result.csv


In [ ]:
# ---------------------------------------------------
# 0. 라이브러리
import os, time, warnings, threading
import concurrent.futures as cf
from pathlib import Path
from datetime import timedelta

import pandas as pd
import FinanceDataReader as fdr
from tqdm.auto import tqdm
warnings.filterwarnings("ignore", category=UserWarning)

# ---------------------------------------------------
# 1. 설정
BASE        = "/Users/gun/Desktop/미래에셋 AI 공모전/data"
div_path    = f"{BASE}/dividend_ml_ready.csv"
hist_path   = f"{BASE}/price_history.csv"
check_path  = f"{BASE}/window_check_result.csv"
fail_path   = f"{BASE}/failed_codes.csv"
cache_dir   = Path(BASE) / "price_cache"
cache_dir.mkdir(exist_ok=True)

WINDOW_DAYS = 30
N_WORKERS   = 8
MAX_RETRY   = 3
BACKOFF     = 2.5

# ---------------------------------------------------
# 2. 공시 이벤트 로드
df_div = pd.read_csv(div_path, dtype={"stock_code": str, "rcept_no": str})
df_div["stock_code"] = df_div["stock_code"].str.zfill(6)
df_div["rcept_dt"] = pd.to_datetime(df_div["rcept_no"].str[:8], format="%Y%m%d", errors="coerce")
df_base = df_div[["stock_code", "rcept_dt"]].dropna().drop_duplicates()

krx_live_codes = set(fdr.StockListing("KRX")["Code"])
df_base = df_base[df_base["stock_code"].isin(krx_live_codes)].reset_index(drop=True)

print(f"✅ 이벤트: {len(df_base):,}  |  종목: {len(df_base['stock_code'].unique()):,}")

# ---------------------------------------------------
# 3. 캐시 유틸
def cache_ok(path, start, end):
    try:
        d = pd.read_csv(path, parse_dates=["date"])
        return d["date"].min() <= pd.to_datetime(start) and d["date"].max() >= pd.to_datetime(end)
    except Exception:
        return False

# ---------------------------------------------------
# 4. 주가 다운로드 함수
lock       = threading.Lock()
failed     = []
price_rows = []

def fetch(code, start, end):
    cache = cache_dir / f"{code}.csv"
    if cache.exists() and cache_ok(cache, start, end):
        return pd.read_csv(cache, parse_dates=["date"])
    delay = 1
    for attempt in range(1, MAX_RETRY + 1):
        try:
            df = fdr.DataReader(f"KRX:{code}", start, end)
        except Exception:
            try:
                df = fdr.DataReader(code, start, end)
            except Exception as e:
                if attempt == MAX_RETRY:
                    with lock:
                        failed.append(code)
                        tqdm.write(f"❌ {code} 실패: {e}")
                    return None
                time.sleep(delay)
                delay *= BACKOFF
                continue
        df = df.reset_index()[["Date", "Close", "Volume"]].rename(columns={"Date": "date", "Close": "close", "Volume": "volume"})
        df["stock_code"] = code
        df.to_csv(cache, index=False)
        return df

# ---------------------------------------------------
# 5. 병렬 수집
codes = df_base["stock_code"].unique()
print("📦 주가 수집 시작 ...")
with cf.ThreadPoolExecutor(max_workers=N_WORKERS) as ex:
    futures = []
    for code in codes:
        dates = df_base.loc[df_base["stock_code"] == code, "rcept_dt"]
        s = (dates.min() - timedelta(days=WINDOW_DAYS)).strftime("%Y-%m-%d")
        e = (dates.max() + timedelta(days=WINDOW_DAYS)).strftime("%Y-%m-%d")
        futures.append(ex.submit(fetch, code, s, e))
    for fut in tqdm(cf.as_completed(futures), total=len(futures)):
        r = fut.result()
        if r is not None:
            price_rows.append(r)

df_price_all = pd.concat(price_rows, ignore_index=True)
print(f"✅ 완료 rows: {len(df_price_all):,}  |  실패 종목: {len(failed)}")
pd.DataFrame({"failed_code": failed}).to_csv(fail_path, index=False)

# ---------------------------------------------------
# 6. price_history.csv 생성
hist_rows = []
for (code, dt), g in tqdm(df_base.groupby(["stock_code", "rcept_dt"]), total=len(df_base), desc="price_history"):
    sub = df_price_all[
        (df_price_all["stock_code"] == code) &
        (df_price_all["date"].between(dt - timedelta(days=WINDOW_DAYS), dt + timedelta(days=WINDOW_DAYS)))
    ].copy()
    sub["rcept_dt"] = dt
    hist_rows.append(sub)

df_hist = (
    pd.concat(hist_rows, ignore_index=True)
      .loc[:, ["stock_code", "rcept_dt", "date", "close", "volume"]]
      .sort_values(["stock_code", "rcept_dt", "date"])
)
df_hist.to_csv(hist_path, index=False, encoding="utf-8-sig")
print(f"📁 price_history.csv 저장 ({len(df_hist):,} rows)")

# ---------------------------------------------------
# 7. ±10 거래일 윈도우 검증 및 n_days ≥ 21 필터
def win_len(g, dt, w=10):
    g = g.sort_values("date").reset_index(drop=True)
    pos = g["date"].searchsorted(dt)
    return len(g.iloc[max(pos-w, 0):pos+w+1])

chk = []
for (code, dt), grp in df_hist.groupby(["stock_code", "rcept_dt"]):
    chk.append({"stock_code": code, "rcept_dt": dt, "n_days": win_len(grp, dt)})
df_chk = pd.DataFrame(chk)

# 🔥 기존 파일에 n_days < 21 제거하고 덮어쓰기
df_chk = df_chk[df_chk["n_days"] >= 21].reset_index(drop=True)
df_chk.to_csv(check_path, index=False, encoding="utf-8-sig")
print(f"📁 window_check_result.csv 저장 (n_days ≥ 21 필터 완료)")

print("\n🎉 파이프라인 완료!")

✅ 이벤트: 15,460  |  종목: 1,682
📦 주가 수집 시작 ...


100%|██████████| 1682/1682 [00:43<00:00, 38.89it/s]


✅ 완료 rows: 3,203,069  |  실패 종목: 0


price_history: 100%|██████████| 15460/15460 [18:44<00:00, 13.74it/s]


📁 price_history.csv 저장 (555,328 rows)
📁 window_check_result.csv 저장

🎉 파이프라인 완료!
